# **Importing Necessary Libraries**

In [1]:
import numpy as np

In [2]:
from utils.utils import load_json, write_json
from utils.similarity_score import cosine, match, manhattan_distance

In [3]:
songs = load_json('data/vector_data.json')

# **Creating the Similarity Function**

1. in_movie - `match`
2. lyrics -
3. language - `cosine`
4. topics -
5. sentiment - 
6. artist - `match`
7. album_type - `match`
8. album_name - `match`
9. album_artist - `match percentage`
10. popularity - 
11. age - `scaled manhattan distance`
12. explicit - `match`
13. duration - `scaled manhattan distance`
14. listener_play_ratio - 
15. genre - `cosine`
16. acousticness - `manhattan distance`
17. danceability - `manhattan distance`
18. energy - `manhattan distance`
19. instrumentalness - `manhattan distance`
20. key - `match`
21. liveness - `manhattan distance`
22. loudness - `scaled manhattan distance`
23. mode - `match`
24. speechiness - `manhattan distance`
25. tempo - `scaled manhattan distance`
26. valence - `manhattan distance`
27. num_sections - `scaled manhattan distance`
28. num_segments - `scaled manhattan distance`
29. artist_vector - `cosine`

In [4]:
songs[0]['sentiment']

[1.600017355803449,
 0.05191584728880194,
 -1.2059578394556674,
 0.5469378656502659,
 2.343592311540182,
 -2.1471144160397957,
 -1.4471364167561456,
 -3.283849526369881,
 2.202246195912256,
 1.339348622426536]

In [48]:
songs[3]['duration']

180000

In [16]:
manhattan_distance(songs[4]['acousticness'], songs[5]['acousticness'])

0.754

In [16]:
songs[1]['popularity']

0

In [4]:
function_map = {
    'in_movie': match,
    'lyrics': cosine,
    'language': cosine,
    'topics': cosine,
    'sentiment': cosine,
    'title': match,
    'artist': match,
    'album_type': match,
    'album_name': match,
    'album_artist': match,
    'popularity': cosine,
    'age': cosine,
    'explicit': match,
    'duration': cosine,
    'listener_play_ratio': cosine,
    'genre': cosine,
    'acousticness': manhattan_distance,
    'danceability': manhattan_distance,
    'energy': manhattan_distance,
    'instrumentalness': manhattan_distance,
    'key': match,
    'liveness': manhattan_distance,
    'loudness': cosine,
    'mode': match,
    'speechiness': manhattan_distance,
    'tempo': cosine,
    'valence': manhattan_distance,
    'num_sections': cosine,
    'num_segments': cosine,
    'artist_vector': cosine
}

In [23]:
def similarity(song1, song2):
    global function_map
    scores = []
    features = set(song1.keys()).intersection(song2.keys())

    for feature in features:
        func = function_map[feature]
        score = func(song1[feature], song2[feature])
        
        print(f'{feature}: {score}')
        scores.append(score)
    
    print()
    sim = sum(scores) / len(scores)
    
    return sim

In [26]:
similarity(songs[0], songs[3])

in_movie: 1
tempo: 1.0
valence: 0.54
acousticness: 0.99895
artist_vector: 0.6984161796257901
duration: 0.5185014242152466
instrumentalness: 1
artist: 0
album_name: 0
album_artist: 0.0
mode: 1
loudness: 1.0
num_sections: 1.0
title: 0
genre: 0.6166357546558455
album_type: 1
lyrics: 0.9995553103700101
popularity: 1.0
speechiness: 0.9937
danceability: 0.65
sentiment: 0.7920489864737359
energy: 0.849
key: 0
listener_play_ratio: 1.0
liveness: 0.9675
age: 1.0
explicit: 1
topics: 0.9253543310044039
language: 1.0
num_segments: 1.0



0.7516553995448344